### Tecnolgías del Lenguaje. Entregable 4
---
#### Reducción basada en similitud

Se aplica un **umbral de similitud** para asegurar que solo se conserven aquellos posts que realmente están relacionados con el rasgo psicológico que se está analizando. Las similitudes bajas o negativas indican que el contenido del post no guarda relación significativa con el vector representante del rasgo, por lo que incluirlos introduciría ruido y debilitaría la interpretación. Al mantener únicamente los posts con una similitud superior a 0.3, se garantiza que el conjunto final esté compuesto por ejemplos más coherentes, relevantes y explicativos, lo que permite obtener conclusiones más fiables sobre cómo se manifiesta cada rasgo en el lenguaje de los usuarios. Después, comprueba que por cada combinación de usuario, rasgo y polaridad existe al menos un post y si no quedó ninguno tras el filtro, rescata del archivo original el de mayor similitud, asegurando que nunca se pierde ese caso.

El resultado es un archivo reducido e informativo, almacenado en `material/top50_posts_per_user_reduced.csv`

In [2]:
import pandas as pd

# Cargar el archivo original
df = pd.read_csv("material/top50_posts_per_user.csv")

# Umbral
THRESHOLD = 0.3

# 1️⃣ Filtrar por similarity > 0.3
df_filtrado = df[df["similarity"] > THRESHOLD]

# 2️⃣ Para cada usuario–rasgo–polaridad, asegurar al menos un post
def ensure_minimum_posts(group):
    if len(group) > 0:
        return group
    else:
        # Recuperar todos los posts originales de ese usuario–rasgo–polaridad
        original_group = df.loc[
            (df["username"] == group.name[0]) &
            (df["trait"] == group.name[1]) &
            (df["polarity"] == group.name[2])
        ]
        # Seleccionar el post con mayor similitud
        return original_group.nlargest(1, "similarity")

# Agrupar por las claves relevantes
df_garantizado = (
    df_filtrado
    .groupby(["username", "trait", "polarity"], group_keys=False)
    .apply(ensure_minimum_posts)
)

# Guardar el archivo final
df_garantizado.to_csv("material/top50_posts_per_user_reduced.csv", index=False)

print("Filtrado completado. Posts restantes:", len(df_garantizado))


C:\Users\Usuario\AppData\Local\Temp\ipykernel_15228\4224208564.py:29: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(ensure_minimum_posts)


Filtrado completado. Posts restantes: 140315
